In [28]:
from scipy.stats import norm
import numpy as np
import yfinance as yf
import pandas as pd
from ta.volatility import AverageTrueRange
from datetime import datetime

infl=pd.read_csv('infl.csv').week_of_year
#print(infl)
def get_historical_data(stk):
    df = yf.download(stk,period='max', interval="1wk").dropna()
    df['year']=df.index.isocalendar().year.astype(int)
    df["week_of_year"] = df.index.isocalendar().week + df.index.isocalendar().year * 100
    return df


def trade_parameters(df, yield_perc):
    df["strike_price"] = df.Open.round(0)
    df["straddle_yield"] = yield_perc
    df["straddle_premium"] = df.strike_price * yield_perc
    
    df['ATR']=AverageTrueRange(high=df.High,low=df.Low,close=df.Close,window=5,fillna=False).average_true_range().rank(pct=True).shift(1)
    df['abs_change']=abs(df.Close-df.Open)
    df['abs_change_pct']=df.abs_change/df.Open
    df["profit_loss"] = df.straddle_premium - df.abs_change
    df["worst"] = df.straddle_premium - df.abs_change
    df["log_return"] = np.log((df.strike_price + df.profit_loss) / df.strike_price)
    df['pos_ret']=np.where(df.log_return>0,df.log_return,0)
    df['neg_ret']=np.where(df.log_return<0,df.log_return,0)
    df = df.iloc[1: , ::]
    df=df[df.week_of_year.isin(infl)]
    #print(df)
    lim=0.85
    #df=df[df.ATR<lim]
    
    return df

def annual_group(df):
    df = df.groupby(["year"]).agg(
        {"log_return": "sum",'pos_ret':'sum','neg_ret':'sum','profit_loss':'mean','worst':'min'}
    )
    df['sharpe']=df.pos_ret/abs(df.neg_ret)
    df['recovery_periods']=(-df.worst/df.profit_loss)
    df['recovery_periods']=df['recovery_periods'].mask(df['recovery_periods']<0,0)
    return(df)
    
    
atm_option_data = pd.read_csv("historical_straddle_data/option_data_2022-09-02_2022-08-29.csv")

#print(blank)
arr = []
limited=['SPY']
atm_option_data['yield_perc']=(atm_option_data.atm_call_last + atm_option_data.atm_put_last) / atm_option_data.strike
atm_option_data=atm_option_data[atm_option_data.symbol.isin(limited)]
for i,j in zip(atm_option_data.symbol,atm_option_data.yield_perc):
    
    stock = i
    yield_perc = j
    df = get_historical_data(stock)
    #print(df.tail)
    df = trade_parameters(df, yield_perc)
    #print(df.tail)
    #df.to_csv(stock + ".csv")
    df = annual_group(df)

    #df=df.iloc[::,[0,3,4]]
    #print(df)
    
    years=np.arange(df.index[0], 2023, 1, dtype=int)
    zeros=np.zeros((len(years),len(df.columns)),dtype=float)
    blank=pd.DataFrame(columns=df.columns.values,index=years,data=zeros)

    df=df.replace(float('inf'),500)
    df.to_csv(stock + ".csv")
    print(j)
    print(df)
    lr_avg=(min(
        np.average(df.log_return),
        np.percentile(df.sharpe,50)
    ))

    sharpe_avg=(min(
        np.percentile(df.sharpe,50),
          np.average(df.sharpe)
    ))
    lr_check=len(df.log_return[df.log_return>0])/len(df.log_return)
    sharpe_check=len(df.sharpe[df.sharpe>2])/len(df.sharpe)
    recovery_check=len(df.recovery_periods[df.recovery_periods<7])/len(df.recovery_periods)
    lr_sum=np.sum(df.log_return)
    
    df1=blank.add(df,fill_value=0)
    periods_traded=len(df1.log_return[df1.log_return != 0])/len(df1.log_return)
    
    out=[stock,lr_avg,sharpe_avg,lr_sum,lr_check,sharpe_check,recovery_check,periods_traded]

    arr.append(out)
    #print(arr)
    df.to_csv('out.csv')
finaldf=pd.DataFrame(data=arr,columns=['stock','lr_avg','sharpe_avg','lr_sum','lr_check','sharpe_check','recovery_check','periods_traded'])
print(finaldf)
finaldf.to_csv('weekly_mass_bt.csv')

[*********************100%***********************]  1 of 1 completed
0.023407407407407408
      log_return   pos_ret   neg_ret  profit_loss      worst      sharpe  \
year                                                                       
2013    0.507099  0.516360 -0.009261     1.997288  -1.381171   55.756772   
2014    0.613918  0.654008 -0.040091     2.602574  -3.899395   16.313271   
2015    0.380404  0.495344 -0.114940     1.847465  -7.412808    4.309589   
2016    0.501056  0.552838 -0.051782     2.437107  -3.888526   10.676206   
2017    0.692983  0.692983  0.000000     4.065737   1.620506  500.000000   
2018    0.222252  0.468503 -0.246250     1.404581 -12.637478    1.902545   
2019    0.510733  0.551572 -0.040839     3.464709  -5.902959   13.506116   
2020   -0.093758  0.337251 -0.431008    -0.181556 -19.893107    0.782469   
2021    0.447816  0.497867 -0.050051     4.010109  -5.493032    9.947176   
2022    0.040169  0.261071 -0.220901     0.609614 -15.265937    1.181843  

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == "":
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste